In [ ]:
# Copyright 2025 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Agent Engine in Vertex AI - orchestration

## Init Vertex AI SDK

In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform[agent_engines,adk,langchain,ag2,llama_index]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.2/734.2 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

#### Note: <font color="red">Need to restart the kernel after installing</font>

In [ ]:
# Note:  Need to restart the kernel
%pip install --upgrade --user --quiet langchain-google-vertexai

In [ ]:
exit()

In [ ]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL_NAME = "gemini-2.0-flash"
BUCKET_URI = f"gs://agent-0417"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Configuration and initialization

### Initialize Vertex AI

In [ ]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0417/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0417' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [ ]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

In [ ]:
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

In [ ]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Build a LangGraph Agent.

In [ ]:
#@title Customize orchestration with LangGraph.

# Use runnable interface of Langchain.
# https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/develop/langgraph#orchestration
# https://python.langchain.com/v0.2/docs/concepts/#runnable-interface

from vertexai import agent_engines

def langgraph_builder(*, model, **kwargs):
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser
    from langgraph.graph import END, MessageGraph

    output_parser = StrOutputParser()

    planner = ChatPromptTemplate.from_template(
        "Generate an argument about: {input}"
    ) | model | output_parser

    pros = ChatPromptTemplate.from_template(
        "List the positive aspects of {input}"
    ) | model | output_parser

    cons = ChatPromptTemplate.from_template(
        "List the negative aspects of {input}"
    ) | model | output_parser

    summary = ChatPromptTemplate.from_template(
        "Input:{input}\nGenerate a final response given the critique",
    ) | model | output_parser

    builder = MessageGraph()
    builder.add_node("planner", planner)
    builder.add_node("pros", pros)
    builder.add_node("cons", cons)
    builder.add_node("summary", summary)

    builder.add_edge("planner", "pros")
    builder.add_edge("planner", "cons")
    builder.add_edge("pros", "summary")
    builder.add_edge("cons", "summary")
    builder.add_edge("summary", END)
    builder.set_entry_point("planner")

    return builder.compile()


In [ ]:
  #@title Crate a tool for Google search.
  from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

  google_search_tool = Tool(
      google_search = GoogleSearch()
  )

In [ ]:
#@title Build a Langgraph Agent.

from vertexai import agent_engines

agent = agent_engines.LanggraphAgent(
    model=MODEL_NAME,
    tools=[google_search_tool],
    runnable_builder=langgraph_builder,
)

In [ ]:
#@title Query to an agent.
query = "생성형 AI의 대해서 설명해주세요."

response = agent.query(input={"role": "user", "content": query})

for re in response:
  display(Markdown((re['kwargs']['content'])))
  print('----------------------------------------------------------')


생성형 AI의 대해서 설명해주세요.

----------------------------------------------------------


## 생성형 AI는 혁신적인 도구이지만, 잠재적 위험과 윤리적 고려 사항에 대한 심각한 논의가 필요합니다.

생성형 AI는 텍스트, 이미지, 오디오, 비디오 등 다양한 유형의 콘텐츠를 생성할 수 있는 인공지능 기술입니다. 최근 몇 년 동안 급속도로 발전하여, 단순히 재미있는 장난감 수준을 넘어 예술, 디자인, 마케팅, 교육 등 다양한 분야에서 혁신적인 가능성을 제시하고 있습니다. 예를 들어, 작곡가가 직접 연주하지 않고도 생성형 AI를 통해 새로운 음악을 창작하거나, 마케터가 타겟 고객층에 맞는 광고 문구를 자동 생성할 수 있습니다. 이처럼 생성형 AI는 생산성을 향상시키고, 창의적인 작업을 지원하며, 새로운 비즈니스 모델을 창출할 수 있는 엄청난 잠재력을 가지고 있습니다.

그러나 생성형 AI의 급격한 발전과 보급은 간과할 수 없는 심각한 위험과 윤리적 문제를 야기합니다.

**첫째, 허위 정보 및 가짜 뉴스 확산의 위험이 높아집니다.** 생성형 AI는 매우 정교한 가짜 콘텐츠를 만들 수 있으며, 이는 쉽게 진실과 구별하기 어려워 사회적 혼란과 갈등을 야기할 수 있습니다. 정치적 선전, 사기, 명예훼손 등 다양한 악의적인 목적으로 악용될 가능성이 높으며, 이는 민주주의 사회의 근간을 흔들 수 있습니다.

**둘째, 저작권 침해 및 지적 재산권 보호 문제의 심각성을 더합니다.** 생성형 AI는 기존의 콘텐츠를 학습하여 새로운 콘텐츠를 생성하는데, 이 과정에서 저작권 침해 문제가 발생할 수 있습니다. 특히, 예술 작품이나 음악 등 창작성이 높은 콘텐츠의 경우, 원작자의 권리를 침해하는 경우가 발생할 수 있으며, 이에 대한 명확한 법적 규제가 미비한 상황입니다.

**셋째, 일자리 감소 및 경제적 불평등 심화의 가능성을 내포하고 있습니다.** 생성형 AI는 인간의 노동력을 대체할 수 있는 능력을 가지고 있습니다. 특히, 단순 반복적인 작업뿐만 아니라, 창의적인 작업까지도 자동화할 수 있어, 예술가, 작가, 디자이너 등 다양한 직업군에서 일자리 감소가 예상됩니다. 이는 사회 전체의 경제적 불평등을 심화시키고 사회 불안을 야기할 수 있습니다.

**넷째, 편향된 데이터 학습으로 인한 차별 및 불공정 문제가 발생할 수 있습니다.** 생성형 AI는 학습 데이터에 따라 결과가 크게 달라지는데, 만약 학습 데이터에 편향된 정보가 포함되어 있다면, 생성되는 결과 또한 편향될 수 있습니다. 이는 특정 인종, 성별, 계층에 대한 차별을 강화하거나 불공정한 결과를 초래할 수 있습니다.

따라서 생성형 AI의 긍정적인 측면을 활용하면서도 이러한 위험을 최소화하기 위해서는 기술 개발과 함께 윤리적, 법적, 사회적 측면에서의 심도 있는 논의와 준비가 필수적입니다. 명확한 법적 규제를 마련하고, 윤리적 가이드라인을 개발하며, 사회 구성원 모두가 생성형 AI의 잠재적 위험에 대한 인식을 높여야 합니다. 또한, 일자리 감소 문제에 대한 대비책을 마련하고, 편향된 데이터 학습 문제를 해결하기 위한 노력을 지속해야 합니다.

결론적으로, 생성형 AI는 엄청난 잠재력을 가진 기술이지만, 그 위험과 윤리적 문제에 대한 심각한 고려 없이 무분별하게 사용될 경우, 사회에 부정적인 영향을 미칠 수 있습니다. 따라서 우리는 생성형 AI를 책임감 있고 윤리적으로 사용하기 위한 노력을 기울여야 하며, 이를 통해 기술 혁신과 사회적 안전의 균형을 이루어야 합니다.


----------------------------------------------------------


제시된 글에서 언급된 생성형 AI의 부정적인 측면은 다음과 같습니다.

*   **허위 정보 및 가짜 뉴스 확산의 위험 증가:** 생성형 AI가 정교한 가짜 콘텐츠를 쉽게 만들어 사회적 혼란과 갈등을 야기할 수 있으며, 악의적인 목적으로 악용될 가능성이 높습니다.
*   **저작권 침해 및 지적 재산권 보호 문제 심화:** 생성형 AI가 기존 콘텐츠를 학습하여 새로운 콘텐츠를 생성하는 과정에서 저작권 침해 문제가 발생할 수 있으며, 이에 대한 법적 규제가 미비합니다.
*   **일자리 감소 및 경제적 불평등 심화 가능성:** 생성형 AI가 인간의 노동력을 대체하여 다양한 직업군에서 일자리 감소를 야기하고, 사회 전체의 경제적 불평등을 심화시킬 수 있습니다.
*   **편향된 데이터 학습으로 인한 차별 및 불공정 문제 발생:** 생성형 AI가 학습 데이터에 편향된 정보가 포함되어 있다면, 생성되는 결과 또한 편향되어 특정 인종, 성별, 계층에 대한 차별을 강화하거나 불공정한 결과를 초래할 수 있습니다.

----------------------------------------------------------


제공된 텍스트에 따르면, 생성형 AI의 긍정적인 측면은 다음과 같습니다.

*   **생산성 향상:** 생성형 AI는 작업을 자동화하고, 효율성을 높여 생산성을 크게 향상시킬 수 있습니다.
*   **창의적인 작업 지원:** 작곡, 디자인 등 창의적인 분야에서 새로운 아이디어를 제공하고, 작업 과정을 도와 창작 활동을 지원합니다.
*   **새로운 비즈니스 모델 창출:** 혁신적인 콘텐츠 생성 능력을 바탕으로 새로운 서비스와 시장을 개척하고, 비즈니스 모델을 창출할 수 있습니다.
*   **다양한 분야에서의 혁신적인 가능성 제시:** 예술, 디자인, 마케팅, 교육 등 다양한 분야에서 혁신적인 변화를 가져올 잠재력을 가지고 있습니다.

----------------------------------------------------------


생성형 AI는 텍스트, 이미지, 오디오, 비디오 등 다양한 형태의 콘텐츠를 만들어내는 인공지능 기술입니다. 최근 눈부신 발전을 거듭하며 예술, 디자인, 마케팅, 교육 등 여러 분야에서 혁신적인 가능성을 보여주고 있습니다. 예를 들어, 작곡가가 직접 연주하지 않아도 AI가 새로운 음악을 창작하거나, 마케터가 타겟 고객에 맞는 광고 문구를 자동으로 생성하는 것이 가능합니다. 이러한 능력은 생산성을 높이고 창의적인 작업을 지원하며, 새로운 비즈니스 모델을 만들어내는 데 기여할 수 있습니다.

**긍정적인 측면:**

*   **생산성 향상:** 작업 자동화 및 효율성 증대를 통해 생산성을 크게 향상시킵니다.
*   **창의적인 작업 지원:** 새로운 아이디어를 제공하고 창작 과정을 도와 예술, 디자인 등 다양한 분야에서 창작 활동을 지원합니다.
*   **새로운 비즈니스 모델 창출:** 혁신적인 콘텐츠 생성 능력을 바탕으로 새로운 서비스와 시장을 개척하고 비즈니스 모델을 창출합니다.
*   **다양한 분야에서의 혁신적인 가능성 제시:** 예술, 디자인, 마케팅, 교육 등 다양한 분야에서 혁신적인 변화를 가져올 잠재력을 가지고 있습니다.

**부정적인 측면 및 윤리적 고려 사항:**

하지만 생성형 AI의 발전은 간과할 수 없는 심각한 위험과 윤리적 문제를 동반합니다.

*   **허위 정보 및 가짜 뉴스 확산 위험:** 정교한 가짜 콘텐츠를 쉽게 만들어 사회적 혼란과 갈등을 야기하고, 정치적 선전, 사기, 명예훼손 등 악의적인 목적으로 악용될 수 있습니다.
*   **저작권 침해 및 지적 재산권 보호 문제 심화:** 기존 콘텐츠를 학습하여 새로운 콘텐츠를 생성하는 과정에서 저작권 침해 문제가 발생할 수 있으며, 이에 대한 명확한 법적 규제가 미비합니다.
*   **일자리 감소 및 경제적 불평등 심화 가능성:** 인간의 노동력을 대체하여 다양한 직업군에서 일자리 감소를 야기하고 사회 전체의 경제적 불평등을 심화시킬 수 있습니다.
*   **편향된 데이터 학습으로 인한 차별 및 불공정 문제 발생:** 학습 데이터에 편향된 정보가 포함되어 있다면 생성 결과 또한 편향되어 특정 인종, 성별, 계층에 대한 차별을 강화하거나 불공정한 결과를 초래할 수 있습니다.

**결론:**

생성형 AI는 분명 엄청난 잠재력을 가진 기술이지만, 동시에 여러 위험과 윤리적 문제점을 안고 있습니다. 따라서 기술 개발과 더불어 법적 규제 마련, 윤리적 가이드라인 개발, 사회 구성원의 인식 제고 등 심도 있는 논의와 준비가 필수적입니다. 또한, 일자리 감소 문제에 대한 대비책 마련과 편향된 데이터 학습 문제 해결을 위한 지속적인 노력이 필요합니다. 생성형 AI를 책임감 있고 윤리적으로 사용하여 기술 혁신과 사회적 안전의 균형을 이루는 것이 중요합니다.


----------------------------------------------------------


In [ ]:
print(response)


[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': '생성형 AI의 대해서 설명해주세요.', 'type': 'human', 'id': '0d65a545-4a04-4d31-82f7-6848568ccc92'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': '## 생성형 AI는 혁신적인 도구이지만, 잠재적 위험과 윤리적 고려 사항에 대한 심각한 논의가 필요합니다.\n\n생성형 AI는 텍스트, 이미지, 오디오, 비디오 등 다양한 유형의 콘텐츠를 생성할 수 있는 인공지능 기술입니다. 최근 몇 년 동안 급속도로 발전하여, 단순히 재미있는 장난감 수준을 넘어 예술, 디자인, 마케팅, 교육 등 다양한 분야에서 혁신적인 가능성을 제시하고 있습니다. 예를 들어, 작곡가가 직접 연주하지 않고도 생성형 AI를 통해 새로운 음악을 창작하거나, 마케터가 타겟 고객층에 맞는 광고 문구를 자동 생성할 수 있습니다. 이처럼 생성형 AI는 생산성을 향상시키고, 창의적인 작업을 지원하며, 새로운 비즈니스 모델을 창출할 수 있는 엄청난 잠재력을 가지고 있습니다.\n\n그러나 생성형 AI의 급격한 발전과 보급은 간과할 수 없는 심각한 위험과 윤리적 문제를 야기합니다.\n\n**첫째, 허위 정보 및 가짜 뉴스 확산의 위험이 높아집니다.** 생성형 AI는 매우 정교한 가짜 콘텐츠를 만들 수 있으며, 이는 쉽게 진실과 구별하기 어려워 사회적 혼란과 갈등을 야기할 수 있습니다. 정치적 선전, 사기, 명예훼손 등 다양한 악의적인 목적으로 악용될 가능성이 높으며, 이는 민주주의 사회의 근간을 흔들 수 있습니다.\n\n**둘째, 저작권 침해 및 지적 재산권 보호 문제의 심각성을 더합니다.** 생성형 

## End of Notebook